# Questao 1
O conjunto de microdados do ENEM 2022 possui 3.476.105 linhas, uma numerosidade muito grande para nosso contexto. Sua primeira tarefa é fazer a leitura do arquivo CSV por blocos e então realizar filtragem e amostragem em cada bloco para diminuir a numerosidade, salvando ao final um CSV com numerosidade reduzida. Tente alcançar uma tabela entre 50mil e 100mil linhas.
Para leitura por blocos, use a opção chunksize de read_csv

In [2]:
import pandas as pd


db = pd.read_csv('micro_dados/MICRODADOS_ENEM_2022.csv', encoding="cp1252",delimiter=";")
db_ba = db[ db["SG_UF_ESC"]=="BA" ]
db_mg = db[ db["SG_UF_ESC"]=="MG" ]


In [3]:
print(len(db_ba))
print(db_ba.shape)

44483
(44483, 76)


In [4]:
print(len(db_mg))
print(db_mg.shape)

68869
(68869, 76)


In [5]:
db_ba.to_csv("base_dados_ma_ba.csv",encoding="cp1252")

In [ ]:
db_mg.to_csv("base_dados_ma_mg.csv",encoding="cp1252")

Load data

In [2]:
import pandas as pd
db = pd.read_csv('base_dados.csv', encoding="cp1252")
db.columns

Index(['Unnamed: 0', 'NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', '

# Questão 2

Com base no conjunto de dados reduzido, crie um subconjunto de dados, que será utilizado para predição da faixa de notas de uma prova do ENEM.

## a)
Para este subconjunto escolha um atributo de nota de uma das provas do ENEM, selecione também 10 outros atributos não relacionados às provas (não pertencentes aos atributos da seção Dados da Prova Objetiva nem da seção Dados da Redação). 


resp: idade, tipo escola, Q006,Q024,Q025,sexo, estado civil,Q021, TP_ST_CONCLUSAO,Q005

pq escolheu cada um, pode colocar em bloco de atributo,


## b)

Para o atributo de nota da prova escolhida, crie um novo atributo com a discretização da nota em 4 quartis (apresente os valores delimitantes), e seleciona somente as linhas pertencentes ao primeiro e ao quarto quartil. 


In [38]:
dados = db['NU_NOTA_REDACAO'].describe()
dados = dados['25%'],dados['75%']
colunas = [ "TP_FAIXA_ETARIA", 'TP_ENSINO', 'Q006','Q024','Q025','TP_SEXO', 'TP_ESTADO_CIVIL','Q021', 'TP_ST_CONCLUSAO','Q005']
db_ = db[((db['NU_NOTA_REDACAO'] != 0) & (db['NU_NOTA_REDACAO'] < dados[0])) | (db['NU_NOTA_REDACAO'] > dados[1])].get(colunas).copy()
db_['result'] = (db['NU_NOTA_REDACAO'] > dados[1]) *1
db_['result']

6        0
8        1
10       0
11       0
15       0
        ..
54584    0
54585    1
54588    0
54595    0
54596    1
Name: result, Length: 17091, dtype: int32

## c)

Analise os demais atributos, verifique os valores existentes e sua distribuição, verifique a existência de dados ausentes. Determine e realize os procedimentos de pré-processamento necessários. Pode ser necessário transformar atributos categóricos (por exemplo, transformando alguns valores categóricos em um valor único, ou aplicando one-hot-encoding). Pode ser necessário normalizar ou padronizar atributos. Pode haver demanda de imputar valores ou excluir linhas com dados ausentes. Você pode também criar/transformar outros atributos (por exemplo, discretizando valores numéricos, combinando dois atributos em um único atributo).


In [39]:
db_.dropna(inplace=True)
db_['result'].sum(),db_.shape

(3173, (5332, 11))

In [40]:
db_['TP_SEXO'] = (db_['TP_SEXO'] == 'M')* 1
db_['Q021'] = (db_['Q021'] == 'A')* 1

In [41]:
db_

,TP_FAIXA_ETARIA,TP_ENSINO,Q006,Q024,Q025,TP_SEXO,TP_ESTADO_CIVIL,Q021,TP_ST_CONCLUSAO,Q005,result
10,3,1.0,B,B,B,0,0,1,2,4,0
15,3,1.0,D,A,B,0,1,1,2,3,0
35,4,1.0,D,B,B,0,1,1,2,5,0
44,3,1.0,H,C,B,1,1,1,2,4,1
55,5,1.0,B,A,B,0,1,0,2,4,0
...,...,...,...,...,...,...,...,...,...,...,...
54557,3,1.0,C,B,B,1,1,1,2,3,1
54559,2,1.0,D,A,B,0,1,1,2,4,1
54579,3,1.0,I,B,B,1,0,0,2,4,0
54580,3,1.0,H,B,B,0,1,0,2,6,0
